# Semillero

### Autor: Federico Picado
### Fecha de última modificación: 28/10/2024
### Descripción: Entrenar y guardar modelos solamente cambiando la semillla.

Este script creara N modelos cambiando solo la semilla y se guardaran en la carpeta especificada, de la siguiente forma :

- `<modelo_nombre>_<numero_seed>.ipynb`

## Parámetro

N_Seeds: Cantidad de semillas aleatoria de 1000 a 100000 elegir.

In [83]:
N_Seeds= 50
mes_train = 202104
mes_test = 202106

## Input

In [84]:
# Ya en el input se ejecutan cosas
import pandas as pd
import numpy as np
import random
import lightgbm as lgb
import optuna
import os

In [85]:
base_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/'# Cambiar
dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01_DQ.csv'#df con las columnas que entrene 

# carpeta donde se guardan los modelos
nombre_carpeta = "modelo_shap" # Cambiar
modelo_nombre= "modelo_shap"# Cambiar

# me traigo la optimizacion del modelo
storage_name = "sqlite:///" + db_path + "optimizacion_LGBM_competencia_01.db"# Cambiar
study_name = "modelo_22"# Cambiar

In [86]:
# creo el study
study_1= optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

study=study_1
best_iter=study_1.best_trial.user_attrs["best_iter"]

# Definir los mejores parámetros del modelo
best_params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'verbose': 0
    #agregue los que su modelo considere
}

[I 2024-10-28 20:44:03,467] Using an existing study with name 'modelo_22' instead of creating a new one.


Esta parte podria estar en procesamiento pero creo que aca se va a poder modificar mejor en el caso que el modelo no sea el mismo

In [87]:
### El modelo que entre yo es un LGBM, creando la clase binaria y separe los datos para entrenamiento
data = pd.read_csv(dataset_path + dataset_file)
df=data.copy()

df['clase_peso'] = 1.0
df['clase_binaria'] = 0
df['clase_binaria'] = np.where(df['clase_ternaria'] == 'CONTINUA', 0, 1)

df.loc[df['clase_binaria'] == 1, 'clase_peso'] = 1.0001

### train. Los datos con los que realize la optimizacion
train_data = df[df['foto_mes'] == mes_train]
X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = train_data['clase_binaria']
w_train = train_data['clase_peso']

C:\Users\Federico\AppData\Local\Temp\ipykernel_1188\599366075.py:2: DtypeWarning: Columns (97) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dataset_path + dataset_file)


## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [88]:
base_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/'# cambiar la ruta
modelos_path = base_path + 'modelos/'
# dentro de modelos_path se creara una carpeta y se guardaran los modelos entrenados

# MAIN

### Funciones 

In [89]:
def entrenar_modelos_con_varias_semillas(best_params, X_train, y_train_binaria, w_train, best_iter, semillas,modelo_nombre):
    """
    Entrena modelos de LightGBM utilizando los mejores parámetros y una lista de semillas.
    
    Parámetros:
    - best_params: dict, contiene los mejores parámetros del estudio de optimización.
    - X_train: dataframe, las características de entrenamiento.
    - y_train_binaria: array-like, la variable objetivo binaria.
    - w_train: array-like, los pesos para las observaciones de entrenamiento.
    - best_iter: int, la mejor cantidad de árboles según el estudio de optimización.
    - semillas: lista de int, las semillas para entrenar cada modelo.
    
    Retorna:
    - modelos_entrenados: diccionario de modelos de LightGBM entrenados, con nombres basados en la semilla.
    """
    modelos_entrenados = {}

    # Crear el dataset de entrenamiento
    train_data = lgb.Dataset(X_train, label=y_train_binaria, weight=w_train)

    # Entrenar un modelo por cada semilla
    for seed in semillas:
        # Actualizar el parámetro de la semilla
        params = best_params.copy()
        params['seed'] = seed
        
        # Entrenar el modelo
        model = lgb.train(params, train_data, num_boost_round=best_iter)
        
        # Asignar el nombre del modelo como "modelo_semilla"
        nombre_modelo = f"{modelo_nombre}_{seed}"
        modelos_entrenados[nombre_modelo] = model
        
        print(f"Modelo entrenado con semilla {seed} guardado como {nombre_modelo}")
    
    return modelos_entrenados


def crear_semillas_primas_aleatorias(N, inicio=1000, fin=100000):
    def es_primo(n):
        if n < 2:
            return False
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0:
                return False
        return True

    semillas = set()  # Usamos un conjunto para evitar duplicados
    while len(semillas) < N:
        num = random.randint(inicio, fin)
        if es_primo(num):
            semillas.add(num)
    return list(semillas)

In [90]:
# Creo las semillas
semillas = crear_semillas_primas_aleatorias(N_Seeds)

In [91]:
# Entrenar los modelos
modelos_entrenado = entrenar_modelos_con_varias_semillas(best_params, X_train, y_train_binaria, w_train, best_iter, semillas, modelo_nombre)

Modelo entrenado con semilla 95233 guardado como modelo_shap_95233
Modelo entrenado con semilla 13697 guardado como modelo_shap_13697
Modelo entrenado con semilla 53381 guardado como modelo_shap_53381
Modelo entrenado con semilla 37643 guardado como modelo_shap_37643
Modelo entrenado con semilla 49807 guardado como modelo_shap_49807
Modelo entrenado con semilla 76303 guardado como modelo_shap_76303
Modelo entrenado con semilla 23059 guardado como modelo_shap_23059
Modelo entrenado con semilla 27541 guardado como modelo_shap_27541
Modelo entrenado con semilla 30869 guardado como modelo_shap_30869
Modelo entrenado con semilla 23321 guardado como modelo_shap_23321
Modelo entrenado con semilla 53147 guardado como modelo_shap_53147
Modelo entrenado con semilla 42139 guardado como modelo_shap_42139
Modelo entrenado con semilla 35999 guardado como modelo_shap_35999
Modelo entrenado con semilla 99487 guardado como modelo_shap_99487
Modelo entrenado con semilla 96419 guardado como modelo_shap_9

In [92]:
def crear_carpeta(base_ruta, nombre_carpeta):
    """
    Crea una carpeta en la ruta especificada si no existe y devuelve la ruta completa.
    
    Parámetros:
    - base_ruta: str, ruta base donde se creará la carpeta.
    - nombre_carpeta: str, nombre de la carpeta a crear.
    
    Retorna:
    - ruta_completa: str, la ruta completa de la carpeta creada.
    """
    # Construye la ruta completa de la carpeta
    ruta_completa = os.path.join(base_ruta, nombre_carpeta)
    
    # Verifica si la carpeta ya existe; si no, la crea
    if not os.path.exists(ruta_completa):
        os.makedirs(ruta_completa)
        print(f"Carpeta creada en: {ruta_completa}")
    else:
        print(f"La carpeta ya existe en: {ruta_completa}")
    
    # Retorna la ruta completa de la carpeta
    return ruta_completa

In [93]:
ruta_modelos=crear_carpeta(modelos_path, nombre_carpeta)

Carpeta creada en: C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/modelos/modelo_shap


In [94]:
def guardar_modelo(model, modelos_carpeta_path, nombre_archivo):
    """
    Guarda un modelo de LightGBM en un archivo especificado.
    
    Parámetros:
    - model: el modelo de LightGBM entrenado.
    - modelos_path: str, la ruta de la carpeta donde se guardará el modelo.
    - nombre_archivo: str, el nombre del archivo (incluyendo la extensión .txt) para guardar el modelo.
    """
    # Asegurar que la carpeta existe
    if not os.path.exists(modelos_carpeta_path):
        os.makedirs(modelos_carpeta_path)
    
    # Construir la ruta completa del archivo
    ruta_completa = os.path.join(modelos_carpeta_path, nombre_archivo)
    
    # Guardar el modelo en la ruta especificada
    model.save_model(ruta_completa)

In [95]:
for nombre_modelo, modelo in modelos_entrenado.items():
    guardar_modelo(modelo, ruta_modelos, f"{nombre_modelo}.txt")